In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd
import requests
from tqdm import tqdm

In [2]:
driver = webdriver.Chrome("../../../chromedriver")
airlines = pd.read_csv("../data/airline_links_tripadvisor.csv")

In [3]:
#Go through all the pages of reviews
for airline in airlines["airlines"]:
    print(airline)
    airline_name = []
    reviews = []
    dates = []
    scores = []
    tags = []
    #Get reviews from airline
    url = "https://www.tripadvisor.com" + airlines.loc[airlines['airlines'] == airline]["links"].values[0]
    driver.get(url)
    #Scroll down page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    time.sleep(2)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    page = BeautifulSoup(driver.page_source)
    try:
        total_pages = int(page.find("div", {"class": "pageNumbers"}).find_all("a")[-1].text)
    except:
        total_pages = 1
    for i in tqdm(range(total_pages)):
        driver.get(url)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        time.sleep(1)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        page = BeautifulSoup(driver.page_source)

        #Review text
        for review in page.find_all("q", {"class": "IRsGHoPm"}):
            reviews.append(review.text)
            airline_name.append(airline)
        
        #Review date
        for date in page.find_all("span", {"class": "_34Xs-BQm"}):
            dates.append(date.text)

        #Review scores
        for score in page.find_all("div", {"class": "nf9vGX55"}):
            scores.append(int(score.find("span").get("class")[1][7]))

        #Trip information
        for tag in page.find_all("div", {"class": "hpZJCN7D"}):
            tags.append([stag.text for stag in tag.find_all("div", {"class": "_3tp-5a1G"})])
        
        if total_pages > 1:
            url = url[:60] + "or" + str((i + 1)*5) + "-" + url[60:]
        
    reviews = pd.DataFrame({"reviews": reviews, "scores": scores, "tags": tags})
    reviews.to_csv("../data/trip_advisor_reviews" + airline + ".csv", sep = ",", index = False)

Advanced Air
100%|██████████| 1/1 [00:01<00:00,  1.66s/it]
AEGEAN
  1%|          | 11/958 [00:18<28:02,  1.78s/it]